In [70]:
import pandas as pd
from typing import List
import geopy.distance

# Arrival

In [71]:
arrival_df = pd.read_excel('Data/arrivals.xlsx', sheet_name=1)
arrival_df.head(1)

,Vessel Name,Port Call Type,Port Type,Port At Call,Port At Call Country,Latitude,Longitude,Ata/atd,Time At Port,Port Country,Port Location,Type,Remarks,Latitude (generated),Longitude (generated),Total Capcity of Methanol Storage ( In Metric Tonnes)
0,SEABOURN VENTURE,DEPARTURE,Port,REYKJAVIK,IS,64.1265,-21.8174,2023-06-04 20:07:00,14h 19m,China,Zhuhai,Confirmed Methanol Supply / Storage,NaN,22.2667,113.5667,3957.0


In [72]:
# Preprocess Arrivals df
arrival_df.rename(columns={
    'Port At Call': 'location', 
    'Port At Call Country': 'country',
    'Latitude': 'lat',
    'Longitude': 'long',
    'Ata/atd': 'time'
    }, inplace=True)
arrival_df['location']=arrival_df.location.map(lambda y: y.title())
arrival_df=arrival_df.sort_values('time', ascending=True).reset_index(drop=True)
arrival_df = arrival_df [['location', 'country', 'lat', 'long', 'time']]
arrival_df.head(1)

,location,country,lat,long,time
0,Genova,IT,44.4056,8.9463,2022-07-01 16:45:00


# Methanol

In [73]:
methanol_df = pd.read_excel('Data/methanol.xlsx')
methanol_df.head(1)

,Port Country,Port Location,Type,Remarks,Latitude (generated),Longitude (generated),Total Capcity of Methanol Storage ( In Metric Tonnes)
0,China,Zhuhai,Confirmed Methanol Supply / Storage,NaN,22.2667,113.5667,3957.0


In [74]:
# Preprocess methanol df
methanol_df.rename(columns={
    'Port Location': 'location', 
    'Port Country': 'country',
    'Latitude (generated)': 'lat',
    'Longitude (generated)': 'long',
    }, inplace=True)
methanol_df = methanol_df [['location', 'country', 'lat', 'long']]
methanol_df.head(1)

,location,country,lat,long
0,Zhuhai,China,22.2667,113.5667


# Compute

In [98]:
def compute_distance(A: pd.DataFrame, B: pd.DataFrame, max_distance: int, radious: int) -> pd.DataFrame:
    result = pd.DataFrame()
    for a_index, Arow in A.iterrows():
        for b_index, Brow in B.iterrows():
            coords_1 = (Arow.lat, Arow.long)
            coords_2 = (Brow.lat, Brow.long)
            dist=(geopy.distance.geodesic(coords_1, coords_2))
            if dist.km<= max_distance*radious:
                new_row = {}
                new_row['arrival_idx'] = a_index
                new_row['arrival_time']= Arow.time
                new_row['arrival_location'] = Arow.location
                new_row['arrival_country'] = Arow.country
                new_row['methanol_hub'] = f'{Brow.country}_{Brow.location}'
                new_row['methanol_location'] = Brow.location
                new_row['methanol_country'] = Brow.country
                new_row['km_distance'] = round(dist.km, 1)
                new_row['nm_distance'] = round(dist.nautical, 1)
                row_df = pd.DataFrame(new_row, index=[0])
                result = pd.concat([result, row_df])
    return result.reset_index(drop=True)

In [99]:
def remove_duplicates(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    return df.sort_values(by=columns, ascending=True).drop_duplicates(subset='arrival_location').sort_values(by='arrival_idx', ascending=True).reset_index(drop=True)

In [100]:
def compute_beetween_stops_analytics(df: pd.DataFrame) -> pd.DataFrame:
    previous_row = {
        "arrival_idx": -1,
        "arrival_time": df.iloc[0].arrival_time
    }
    for index, row in df.iterrows():
        df.at[index, 'stops_in_between'] = row.arrival_idx - previous_row["arrival_idx"] - 1
        df.at[index, 'time_in_between'] = row.arrival_time - previous_row["arrival_time"]

        previous_row["arrival_idx"] = row.arrival_idx
        previous_row["arrival_time"] = row.arrival_time
    
    df['stops_in_between']=df['stops_in_between'].astype(int)
    return df

In [119]:
def compute_travelled_distance(starting_df: pd.DataFrame, final_df: pd.DataFrame)-> pd.DataFrame:
    stops=final_df.arrival_idx.values
    for index, row in starting_df.iterrows():
        if index==0: 
            crr=0
            prev_row = row
            pass
        coords_1 = (prev_row.lat, prev_row.long)
        coords_2 = (row.lat, row.long)
        crr += (geopy.distance.geodesic(coords_1, coords_2)).nautical
        #starting_df.at[index, 'travelled_distance'] = crr
        prev_row=row
        if index in stops:
            final_df.loc[final_df.arrival_idx==index, 'travelled_distance']=round(crr, 2)
            crr=0
    return final_df.drop(columns=['arrival_idx'])
    

### Testing the code

In [116]:
# 1 degree of latitude, called an arc degree, covers about 111 kilometers 
res=compute_distance(arrival_df, methanol_df, 111.12, 1)
res1=remove_duplicates(res, ['arrival_location', 'km_distance'])
res2=compute_beetween_stops_analytics(res1)
res3=compute_travelled_distance(arrival_df,res2)
res3

,arrival_time,arrival_location,arrival_country,methanol_hub,methanol_location,methanol_country,km_distance,nm_distance,stops_in_between,time_in_between,travelled_distance
0,2022-07-01 16:45:00,Genova,IT,Italy_Genoa,Genoa,Italy,1.0,0.5,0,0 days 00:00:00,0.00
1,2022-07-18 15:26:00,Gibraltar,GI,Spain_Algeciras,Algeciras,Spain,9.1,4.9,4,16 days 22:41:00,1551.80
2,2022-07-23 13:17:00,Rotterdam Centrum,NL,Netherlands_Rotterdam,Rotterdam,Netherlands,5.3,2.8,0,4 days 21:51:00,1035.88
3,2022-10-15 23:25:00,Santa Marta,CO,Colombia_Barranquilla,Barranquilla,Colombia,70.9,38.3,24,84 days 10:08:00,8803.94
4,2022-10-27 22:28:00,Callao,PE,Peru_Callao,Callao,Peru,10.5,5.7,4,11 days 23:03:00,2144.31
5,2022-11-07 20:34:00,San Antonio,CL,Chile_Quintero,Quintero,Chile,90.0,48.6,6,10 days 22:06:00,1463.84
6,2022-11-14 08:50:00,Punta Arenas Anch,CL,Chile_Punta Arenas,Punta Arenas,Chile,91.6,49.5,1,6 days 12:16:00,1183.11
7,2022-11-15 21:57:00,Punta Arenas,CL,Chile_Punta Arenas,Punta Arenas,Chile,91.6,49.5,0,1 days 13:07:00,0.00
8,2023-03-14 21:07:00,Santos,BR,Brazil_Santos,Santos,Brazil,0.5,0.3,42,118 days 23:10:00,13309.03
9,2023-05-17 13:45:00,Clydeport Greenock,GB,UK_Grangemouth,Grangemouth,UK,65.8,35.5,19,63 days 16:38:00,9529.37


# Radious 1

In [30]:
res=compute_distance(arrival_df, methanol_df, 111.12, 1)
res = remove_duplicates(res, ['arrival_location', 'km_distance'])
res=res[['arrival_location', 'arrival_country', 'methanol_hub', 'nm_distance']]
res

In [29]:
res.to_csv('arrival_meth_radious1.csv', index=True)

,arrival_location,arrival_country,methanol_hub,nm_distance
0,Genova,IT,Italy_Genoa,0.5
1,Gibraltar,GI,Spain_Algeciras,4.9
2,Rotterdam Centrum,NL,Netherlands_Rotterdam,2.8
3,Santa Marta,CO,Colombia_Barranquilla,38.3
4,Callao,PE,Peru_Callao,5.7
5,San Antonio,CL,Chile_Quintero,48.6
6,Punta Arenas Anch,CL,Chile_Punta Arenas,49.5
7,Punta Arenas,CL,Chile_Punta Arenas,49.5
8,Santos,BR,Brazil_Santos,0.3
9,Clydeport Greenock,GB,UK_Grangemouth,35.5


# Radious 4

In [31]:
res=compute_distance(arrival_df, methanol_df, 111.12, 4)
res = remove_duplicates(res, ['arrival_location', 'km_distance'])
res=res[['arrival_location', 'arrival_country', 'methanol_hub', 'nm_distance']]
res

,arrival_location,arrival_country,methanol_hub,nm_distance
0,Genova,IT,Italy_Genoa,0.5
1,Gibraltar,GI,Spain_Algeciras,4.9
2,Rotterdam Centrum,NL,Netherlands_Rotterdam,2.8
3,Bridgetown,BB,Trinidad & Tobago_Port Lisas,195.0
4,Santa Marta,CO,Colombia_Barranquilla,38.3
5,Callao,PE,Peru_Callao,5.7
6,Pisco,PE,Peru_Callao,112.5
7,Coquimbo,CL,Chile_Quintero,169.7
8,San Antonio,CL,Chile_Quintero,48.6
9,Punta Arenas Anch,CL,Chile_Punta Arenas,49.5


In [33]:
res.to_csv('arrival_meth_radious4.csv', index=True)

# Operational Parameters 

In [117]:
south_america_countries=['CO', 'BR', 'CL', 'PE', 'AR', 'BB']

def compute_distance_enricched(A: pd.DataFrame, B: pd.DataFrame, max_distance: int) -> pd.DataFrame:
    result = pd.DataFrame()
    for a_index, Arow in A.iterrows():
        for b_index, Brow in B.iterrows():
            coords_1 = (Arow.lat, Arow.long)
            coords_2 = (Brow.lat, Brow.long)
            dist=(geopy.distance.geodesic(coords_1, coords_2))
            if ((Arow.country in south_america_countries) and (dist.km<= max_distance*1)) or ((Arow.country not in south_america_countries) and (dist.km<= max_distance*4)): 
                new_row = {}
                new_row['arrival_idx'] = a_index
                new_row['arrival_time']= Arow.time
                new_row['arrival_location'] = Arow.location
                new_row['arrival_country'] = Arow.country
                new_row['methanol_hub'] = f'{Brow.country}_{Brow.location}'
                new_row['methanol_location'] = Brow.location
                new_row['methanol_country'] = Brow.country
                new_row['km_distance'] = round(dist.km, 1)
                new_row['nm_distance'] = round(dist.nautical, 1)
                row_df = pd.DataFrame(new_row, index=[0])
                result = pd.concat([result, row_df])
    return result.reset_index(drop=True)

In [120]:
res=compute_distance_enricched(arrival_df, methanol_df, 111.12)
res = remove_duplicates(res, ['arrival_location', 'km_distance'])
res=compute_beetween_stops_analytics(res)
res=compute_travelled_distance(arrival_df,res)

res

,arrival_time,arrival_location,arrival_country,methanol_hub,methanol_location,methanol_country,km_distance,nm_distance,stops_in_between,time_in_between,travelled_distance
0,2022-07-01 16:45:00,Genova,IT,Italy_Genoa,Genoa,Italy,1.0,0.5,0,0 days 00:00:00,0.00
1,2022-07-18 15:26:00,Gibraltar,GI,Spain_Algeciras,Algeciras,Spain,9.1,4.9,4,16 days 22:41:00,1551.80
2,2022-07-23 13:17:00,Rotterdam Centrum,NL,Netherlands_Rotterdam,Rotterdam,Netherlands,5.3,2.8,0,4 days 21:51:00,1035.88
3,2022-10-15 23:25:00,Santa Marta,CO,Colombia_Barranquilla,Barranquilla,Colombia,70.9,38.3,24,84 days 10:08:00,8803.94
4,2022-10-27 22:28:00,Callao,PE,Peru_Callao,Callao,Peru,10.5,5.7,4,11 days 23:03:00,2144.31
5,2022-11-07 20:34:00,San Antonio,CL,Chile_Quintero,Quintero,Chile,90.0,48.6,6,10 days 22:06:00,1463.84
6,2022-11-14 08:50:00,Punta Arenas Anch,CL,Chile_Punta Arenas,Punta Arenas,Chile,91.6,49.5,1,6 days 12:16:00,1183.11
7,2022-11-15 21:57:00,Punta Arenas,CL,Chile_Punta Arenas,Punta Arenas,Chile,91.6,49.5,0,1 days 13:07:00,0.00
8,2023-03-14 21:07:00,Santos,BR,Brazil_Santos,Santos,Brazil,0.5,0.3,42,118 days 23:10:00,13309.03
9,2023-05-08 07:04:00,Brest,FR,France_La Rochelle,La Rochelle,France,353.8,191.0,13,54 days 09:57:00,8485.60


In [121]:
res.to_csv('operational_parameters.csv', index=False)

# Operational Parameters- Renewable facilities

In [122]:
emethanol_df1 = pd.read_excel('Data/emethanol.xlsx')
emethanol_df1.head(1)

,Latitude,Longtitude,Capacity (t/y),City,Company,Country,Feedstock
0,63.868877,20.411105,100000.0,Umeå,Liquid Wind and Umeå Energi,Sweden,CO2 and H2 from water electrolysis


In [123]:
# Preprocess methanol df
emethanol_df1.rename(columns={
    'City': 'location', 
    'Country': 'country',
    'Latitude': 'lat',
    'Longtitude': 'long',
    }, inplace=True)
emethanol_df1 = emethanol_df1 [['location', 'country', 'lat', 'long']]
emethanol_df1.head(1)

,location,country,lat,long
0,Umeå,Sweden,63.868877,20.411105


In [124]:
emethanol_df2 = pd.read_excel('Data/emethanol_sa.xlsx')
emethanol_df2.head(1)

,Latitude,Longtitude,Capacity (t/y),City,Company,Country,Feedstock,Product,Start-up year,Product.1,Start-up year.1
0,31.9586,-99.9118,80000.0,Texas,"Celanese, Mitsui & Co",USA,Recycled CO2,E-methanol,2022.0,E-methanol,2022.0


In [125]:
# Preprocess methanol df
emethanol_df2.rename(columns={
    'City': 'location', 
    'Country': 'country',
    'Latitude': 'lat',
    'Longtitude': 'long',
    }, inplace=True)
emethanol_df2 = emethanol_df2 [['location', 'country', 'lat', 'long']]
emethanol_df2.head(1)

,location,country,lat,long
0,Texas,USA,31.9586,-99.9118


In [126]:
emethanol_df=pd.concat([emethanol_df1,emethanol_df2]).reset_index(drop=True)
emethanol_df

,location,country,lat,long
0,Umeå,Sweden,63.868877,20.411105
1,Sundsvall,Sweden,62.403937,17.391813
2,Stenungsund,Sweden,58.067800,11.829400
3,Örnsköldsvik,Sweden,63.290000,18.716600
4,Mönsterås,Sweden,57.041600,16.443100
...,...,...,...,...
161,Baton Rouge,USA,30.450700,-91.154600
162,Barranquilla,Colombia,10.963000,-74.796000
163,Alabama,USA,43.096400,-78.390900
164,Los Angeles,USA,34.054400,-118.243900


In [130]:
res=compute_distance_enricched(arrival_df, emethanol_df, 111.12)
res = remove_duplicates(res, ['arrival_location', 'km_distance'])
res=compute_beetween_stops_analytics(res)
res=compute_travelled_distance(arrival_df,res)

res

,arrival_time,arrival_location,arrival_country,methanol_hub,methanol_location,methanol_country,km_distance,nm_distance,stops_in_between,time_in_between,travelled_distance
0,2022-07-01 16:45:00,Genova,IT,Italy_Genoa,Genoa,Italy,1.0,0.5,0,0 days 00:00:00,0.00
1,2022-07-18 15:26:00,Gibraltar,GI,Spain_Algeciras,Algeciras,Spain,9.1,4.9,4,16 days 22:41:00,1551.80
2,2022-07-23 13:17:00,Rotterdam Centrum,NL,Netherlands_Rotterdam,Rotterdam,Netherlands,4.4,2.4,0,4 days 21:51:00,1035.88
3,2022-07-27 15:13:00,Tromso,NO,Norway_Troms and Finnmark,Troms and Finnmark,Norway,170.4,92.0,0,4 days 01:56:00,1141.31
4,2022-07-30 12:35:00,Bjornoya,NO,Norway_Troms and Finnmark,Troms and Finnmark,Norway,513.9,277.5,0,2 days 21:22:00,292.20
5,2022-08-20 19:24:00,Patreksfjordur,IS,Iceland_Grindavík,Grindavík,Iceland,204.8,110.6,5,21 days 06:49:00,1379.54
6,2022-08-21 19:35:00,Reykjavik,IS,Iceland_Grindavík,Grindavík,Iceland,41.1,22.2,0,1 days 00:11:00,104.52
7,2022-08-22 06:41:00,Grundarfjordur Anch,IS,Iceland_Grindavík,Grindavík,Iceland,124.2,67.1,0,0 days 11:06:00,61.32
8,2022-08-22 17:14:00,Grundarfjordur,IS,Iceland_Grindavík,Grindavík,Iceland,124.2,67.1,0,0 days 10:33:00,0.00
9,2022-08-31 21:41:00,Akureyri,IS,Iceland_Húsavík,Húsavík,Iceland,53.1,28.7,0,9 days 04:27:00,137.32


In [131]:
res.to_csv('operational_parameters_renewable_facilities.csv', index=False)

In [ ]:
#index 4 dopo Tromso {'Bjornoya    Troms and Finnmark   513.9405986742166 km'}
#if ((Arow.country in south_america_countries) and (dist.km<= max_distance*1)) or ((Arow.country not in south_america_countries) and (dist.km<= max_distance*4)) or (Arow.location=='Bjornoya' and Brow.location=='Troms and Finnmark'): 
